# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
import logging

import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails

import joblib

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


# Workspace

In [4]:
# config.json downloaded form Azure Envoirment
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-236295
aml-quickstarts-236295
westus2
81cefad3-d2c9-4f77-a466-99a7f541c7bb


# Azure ML Experiment

Let's create an experiment named "heart-faliure-prediction" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

In [5]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'heart-faliure-prediction'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-faliure-prediction,quick-starts-ws-236295,Link to Azure Machine Learning studio,Link to Documentation


# Create or Attach an AmlCompute cluster

Created a compute target for the AutoML run. 

In [6]:
# Choose a name for the cluster
cpu_cluster_name = "demo11"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
The Heart Faliure Dataset is publically available dataset on Kaggle. It consist of data from 299 patients out of which 105 are women and 194 are men with Heart Faliure. The aim of this project is to classify patients based on their odds of survival.

In [43]:
found = False
key = "Heart-Failure-Dataset"
description_text = "Prediction of survival of patients with heart failure"

df = dataset.to_pandas_dataframe()
df.describe()


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


## AutoML Configuration

The following AutoML settings and configrations were used for the AutoMl run:

In [9]:
# Automl settings

automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

# Parameters for AutoMLConfig

automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)
remote_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on demo11 with default configuration
Running on remote compute: demo11


Experiment,Id,Type,Status,Details Page,Docs Page
heart-faliure-prediction,AutoML_553367df-b11b-469e-a776-b952dd82059e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

{'runId': 'AutoML_553367df-b11b-469e-a776-b952dd82059e',
 'target': 'demo11',
 'status': 'Completed',
 'startTimeUtc': '2023-06-21T20:37:25.167731Z',
 'endTimeUtc': '2023-06-21T20:48:32.258114Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'demo11',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-faliure-prediction","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-236295","workspace_name":"quick-starts-ws-236295","region":"westus2","compute_target":"demo11","spark_service

In [11]:
# Fetch the latest status of the run. It should show 'Completed'

print("Run Status: ",remote_run.get_status())

Run Status:  Completed


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()



_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)


Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Run(Experiment: heart-faliure-prediction,
Id: AutoML_553367df-b11b-469e-a776-b952dd82059e_21,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                ('XGBoostClassifier',
                 XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=10, max_leaves=511, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=2.1875, reg_lambda=0.4166666666666667, subsample=0.5, tree_method='auto'))],
         verbose=False)


In [64]:
print('Best Run Id: ', best_run.id) 
print('\n')

Best Run Id:  AutoML_553367df-b11b-469e-a776-b952dd82059e_21




In [65]:
# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
print('\n')

Best run metrics : {'average_precision_score_weighted': 0.9095165577649078, 'AUC_micro': 0.9158074901530961, 'f1_score_macro': 0.8283452668548864, 'AUC_macro': 0.9084764198495914, 'average_precision_score_micro': 0.9154966438339871, 'recall_score_weighted': 0.8561744966442952, 'precision_score_macro': 0.8494700978562384, 'recall_score_micro': 0.8561744966442952, 'accuracy': 0.8561744966442952, 'f1_score_weighted': 0.8529378345375922, 'AUC_weighted': 0.9084764198495915, 'average_precision_score_macro': 0.8853339173064503, 'log_loss': 0.3877487550602232, 'recall_score_macro': 0.8178928934118412, 'matthews_correlation': 0.665886071533573, 'balanced_accuracy': 0.8178928934118412, 'f1_score_micro': 0.8561744966442952, 'precision_score_weighted': 0.8578216701335741, 'precision_score_micro': 0.8561744966442952, 'norm_macro_recall': 0.6357857868236825, 'weighted_accuracy': 0.8859478415426087, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_553367df-b11b-469e-a776-b952dd82059e_2

In [66]:
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('\n')

Best run details : {'runId': 'AutoML_553367df-b11b-469e-a776-b952dd82059e_21', 'target': 'demo11', 'status': 'Completed', 'startTimeUtc': '2023-06-21T20:43:59.04702Z', 'endTimeUtc': '2023-06-21T20:45:11.135282Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '3fd2e115482e0c7b92aa3297688aefb6c3aa6fb8', 'pipeline_spec': '{"objects": [{"class_name": "StandardScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"with_mean": false, "with_std": false}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "XGBoostClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"booster": "gbtree", "colsample_bytree": 1, "eta": 0.3, "gamma": 0, "max_depth": 10, "max_leaves": 511, "n_estimators": 10, "objective": "reg:logistic", "reg_alpha": 2.1875, "reg_lambda": 0.4166666666666667, "subsample": 0.5, "tree_method": "auto"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "3f

In [24]:
# get_properties()
# Fetch the latest properties of the run from the service
print("Best run properties :",best_run.get_properties())
print('\n' * 3)

Best run properties : {'runTemplate': 'automl_child', 'pipeline_id': '3fd2e115482e0c7b92aa3297688aefb6c3aa6fb8', 'pipeline_spec': '{"objects": [{"class_name": "StandardScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"with_mean": false, "with_std": false}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "XGBoostClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"booster": "gbtree", "colsample_bytree": 1, "eta": 0.3, "gamma": 0, "max_depth": 10, "max_leaves": 511, "n_estimators": 10, "objective": "reg:logistic", "reg_alpha": 2.1875, "reg_lambda": 0.4166666666666667, "subsample": 0.5, "tree_method": "auto"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "3fd2e115482e0c7b92aa3297688aefb6c3aa6fb8", "module": "sklearn.pipeline", "class_name": "Pipeline", "pipeline_name": "{ StandardScaler, XGBoostClassifier }"}', 'training_percent': '100', 'predicted_cost': '0.0084079806568242

In [56]:

best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/65972819/classes.interpret.json',
 'explanation/65972819/eval_data_viz.interpret.json',
 'explanation/65972819/expected_values.interpret.json',
 'explanation/65972819/features.interpret.json',
 'explanation/65972819/global_names/0.interpret.json',
 'explanation/65972819/global_rank/0.interpret.json',
 'explanation/65972819/global_values/0.interpret.json',
 'explanation/65972819/local_importance_values.interpret.json',
 'explanation/65972819/per_class_names/0.interpret.json',
 'explanation/65972819/per_class_rank/0.interpret.json',
 'explanation/65972819/per_class_values/0.interpret.json',
 'explanation/65972819/rich_metadata.interpret.json',
 'explanation/65972819/true_ys_viz.interpret.json',
 'explanation/65972819/visualization_dict.interpret.json',
 'explanation/65972819/ys_pred_proba_viz.interpret.json',
 'explanation/65972819/ys_pred_viz.interpret.json',
 'explanation/772e747f/classes.interpret.json',
 'expl

In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
heart-faliure-prediction,AutoML_553367df-b11b-469e-a776-b952dd82059e_21,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [37]:
# Save the best model
best_model = best_run.register_model(model_name = "Automl_best_model", model_path = './outputs/', 
                         properties={'Accuracy': best_run.get_metrics()['accuracy']})
print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-236295', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-236295'), name=AutoMl_best_run, id=AutoMl_best_run:3, version=3, tags={}, properties={'Accuracy': '0.8561744966442952'})


In [ ]:
# Download the model file
best_run.download_file('outputs/model.pkl', 'Automl_best_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [68]:
model = remote_run.register_model(model_name = 'Automl_best_model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-236295', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-236295'), name=Automl_best_model.pkl, id=Automl_best_model.pkl:3, version=3, tags={}, properties={})


In [52]:
# Download the yaml file that includes the environment dependencies
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

#Downloading the scoring File
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [69]:
inference = InferenceConfig(entry_script='score.py',
                                    environment=best_run.get_environment())

# Deploy the model as a ACI WebService
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1, 
                                                auth_enabled=True, 
                                                enable_app_insights=True)


In [72]:
webservice = Model.deploy(ws, "deployed-heartdata-aci",
                          [model], 
                          inference, 
                          deployment_config)
webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-06-22 01:14:50+00:00 Creating Container Registry if not exists.
2023-06-22 01:14:50+00:00 Registering the environment.
2023-06-22 01:14:50+00:00 Use the existing image.
2023-06-22 01:14:51+00:00 Submitting deployment to compute.
2023-06-22 01:14:56+00:00 Checking the status of deployment deployed-heartdata-aci..
2023-06-22 01:16:58+00:00 Checking the status of inference endpoint deployed-heartdata-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [58]:
# For obtaining the service state and relevant information, 
# such as the scoring URI, Swagger URI, and authentication keys

primary, secondary = webservice.get_keys()

print('Service state:', webservice.state) 
print('Service scoring URI:', webservice.scoring_uri) 
print('Service Swagger URI:', webservice.swagger_uri) 
print('Service primary authentication key:', primary)

Service state: Healthy
Service scoring URI: http://356c4c38-456e-4d3f-81b4-ea7cafac5277.westus2.azurecontainer.io/score
Service Swagger URI: http://356c4c38-456e-4d3f-81b4-ea7cafac5277.westus2.azurecontainer.io/swagger.json
Service primary authentication key: YdQ0Ph4qdgyQTYeViAv263FFzrVDspLS


In [60]:
# Sending a request to the deployed web service and test it by consuming the model endpoint
%run endpoint.py

{"result": [1, 1]}



Expected result: [true, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


In [62]:
# Printing the logs
print(webservice.get_logs())

2023-06-21T23:56:39,687445100+00:00 - rsyslog/run 
2023-06-21T23:56:39,692835800+00:00 - gunicorn/run 
2023-06-21T23:56:39,694390900+00:00 | gunicorn/run | 
2023-06-21T23:56:39,699631200+00:00 | gunicorn/run | ###############################################
2023-06-21T23:56:39,704671800+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-06-21T23:56:39,706635800+00:00 | gunicorn/run | ###############################################
2023-06-21T23:56:39,709671000+00:00 | gunicorn/run | 
2023-06-21T23:56:39,711379700+00:00 - nginx/run 
2023-06-21T23:56:39,720712000+00:00 | gunicorn/run | 
2023-06-21T23:56:39,735009500+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230412.v4
2023-06-21T23:56:39,748222100+00:00 | gunicorn/run | 
2023-06-21T23:56:39,758232500+00:00 | gunicorn/run | 
2023-06-21T23:56:39,762636500+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [71]:
webservice.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
